# Video Game Sales Data Analysis 
---
This Jupyter notebook will explore a dataset of international video game sales in order to identify patterns that determine whether a game succeeds or not.


#### Project Sections 
1. Initial Set Up and Data Preparation 
2. General Data Analysis 
3. Analysis By Sales Region 
4. Hypothesis Testing 

### Initial Set Up & Data Preparation 
---

In [42]:
# Import required libraries 
from scipy import stats as st
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

# Read in Data
df = pd.read_csv('games.csv')

#### Data Cleaning 
Part 1: Finding Issues 

In [43]:
# Sample Data 
df.sample(3)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
6928,Pictionary: Ultimate Edition,PS3,2011.0,Misc,0.15,0.06,0.0,0.03,62.0,tbd,E
6307,Sing Party,WiiU,2012.0,Misc,0.13,0.12,0.0,0.02,60.0,5.4,E10+
11713,Need For Speed: Undercover,PC,2008.0,Racing,0.00,0.07,0.0,0.01,65.0,4.9,T


In [44]:
# Check data types and Missing values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [45]:
# Check for duplicate rows 
df.duplicated().sum()

0

In [46]:
# Check for implicit duplicates
df['Platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [47]:
df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan], dtype=object)

In [48]:
df['Rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

Part 2: Fixing Issues  
Upon initial exploration of the data, it has been determined that the following tasks must be carried out in order to prepare the data for analysis: 
- Column names should be made lowercase 
- Columns with object data types should have strings in lowercase
- user_score should be a float data type
- year_of_release should be an int or object
- Replace missing values: 
    - rating: Replace null with 'unrated'
    - critic_score and user_score: Replace null and 'tbd' with median of genre 
    - genre: replace null with 'misc'
    - name: drop null values (this is 0.01% of data)
    - year_of_release: drop rows with null values (this is 1.6% of data)
- Add a column with total sales 


In [49]:
# Make column names lowercase 
df = df.rename(
    columns={
        'Name':'name',
        'Platform':'platform',
        'Year_of_Release':'year_of_release',
        'Genre':'genre',
        'NA_sales':'na_sales',
        'EU_sales':'eu_sales',
        'JP_sales':'jp_sales',
        'Other_sales':'other_sales',
        'Critic_Score':'critic_score',
        'User_Score':'user_score',
        'Rating':'rating'
    }
)

# Make the contents of object datatype columns lowercase 
df['name'] = df['name'].str.lower()
df['platform'] = df['platform'].str.lower()
df['genre'] = df['genre'].str.lower()
df['rating'] = df['rating'].str.lower()

# Verify Changes
df.sample(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
13373,arslan: the warriors of legend,ps3,2015.0,action,0.0,0.0,0.05,0.0,NaN,tbd,t


In [50]:
# Make user_score a float datatype and get rid of 'tbd'
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

# Make year_of_release a an int data type
df["year_of_release"] = df["year_of_release"].astype("Int64")

# Replace null values in ratings column with 'unrated'
df['rating'] = df['rating'].fillna('unrated')

# Replace null values in genre column with 'misc'
df['genre'] = df['genre'].fillna('misc')

# Replace null values in critic_score column with median for genre
grouped_avg = df.groupby('genre')['critic_score'].transform('median')
df['critic_score'] = df['critic_score'].fillna(grouped_avg)

# Replace null values in user_score column with median for genre group
grouped_avg = df.groupby('genre')['user_score'].transform('median')
df['user_score'] = df['user_score'].fillna(grouped_avg)

# Drop rows with null values in the name or year_of_release columns 
df= df.dropna(axis='rows')

In [51]:
# Add a column with total sales 
df['total_sales'] = df['eu_sales'] + df['jp_sales'] + df['na_sales'] + df['other_sales']

In [52]:
# Verify Changes 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16444 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     16444 non-null  float64
 9   user_score       16444 non-null  float64
 10  rating           16444 non-null  object 
 11  total_sales      16444 non-null  float64
dtypes: Int64(1), float64(7), object(4)
memory usage: 1.6+ MB


In [53]:
df.sample(3)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
16435,h2o+: footprints in the sand,ps2,2008,adventure,0.00,0.00,0.01,0.00,66.0,7.6,unrated,0.01
14256,crazy chicken: star karts,ds,2008,racing,0.03,0.00,0.00,0.00,69.0,7.4,e,0.03
9197,7 days to die,ps4,2016,action,0.04,0.08,0.00,0.02,45.0,5.9,m,0.14


### General Data Analysis
---
In this section, the entire dataset will be analyzed in order to answer some questions about global sales. The following questions will be answered: 
- How long does it generally take for new platforms to appear and old ones to fade?
- Which platforms are leading in sales? Which are growing and shrinking?
- Are the differences in global sales of games for each platform significant? What about average sales on various platforms? 
- Is there a correlation between user reviews and sales for the X platform?
- How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?
- Which genres are the most profitable? Which are the least?

#### Is the data for every period significant?
Volume of games released per year will determine if every period in the data set is significant. 

In [54]:
# Plot the total number of games released by year
fig1 = px.histogram(df,
                    x='year_of_release',
                    title='Figure 1: Number of Games Released by Year')
fig1.update_layout(xaxis_title='Release Year', yaxis_title='Number of Games Released')
fig1.show()


It appears that there was a very small number of games being published annually from the begining of the dataset in 1980 up until 1994, with each year showing under 100 games published. Begining in 1994, the number begins a strong and relatively consistant trend upwards from 121 games published in 1994 to a peak of 1,427 games publised in 2008. From there, the number of games published annually begins to trend downwards with the most significant drop being from 1,136 games publised in 2011 to 653 games published in 2012. Based on these findings, it has been determined that data period between 1980 and 1990 is not significant. 

In [55]:
# Filter out the insignificant period 
df_filtered = df.query("year_of_release > 1989")

C:\Users\Leigh\AppData\Local\Temp\ipykernel_14768\632962174.py:2: RuntimeWarning:

Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.



#### How long does it generally take for new platforms to appear and old ones to fade?
To answer this question, sales variation from platform to platform will be analyzed to determine the platforms with the greatest total sales. Distributions will be built based on data from each year in order to analyze the rise and fall of these platforms. 

In [56]:
# Create a df with the sum of all sales grouped by year and platform 
yearly_platform_sales = df_filtered.groupby(['year_of_release','platform'])['total_sales'].sum().reset_index()

# Plot the total sales by year and platform 
fig2 = px.line(yearly_platform_sales, x="year_of_release", y="total_sales", color="platform", line_group="platform", hover_name="platform",
        line_shape="spline", render_mode="svg", title='Figure 2: Yearly Sales by Platform')
fig2.update_layout(xaxis_title='Year', yaxis_title='Total Sales (USD Million)')
fig2.show()

Moderately to highly sucessful platforms rise and fall with in the span of approximately ten years while less successful platforms may emerge and disappear in as little as one year. The only notable exception is PC as a platform, of which game sales span a period of over twenty-five years. This is likely due to the fact that many people have access to computers and there are reasons to own one beyond just gaming. 

#### Which platforms are leading in sales? Which are growing and shrinking?

In [61]:
# Create a df with the sum of all sales grouped by platform 
platform_sales = df_filtered.groupby('platform')['total_sales'].sum().reset_index()

# Plot the total sales by platform
fig3 = px.histogram(platform_sales,
                    x='platform',
                    y='total_sales',
                    title='Figure 3: Total Sales by Platform',
                    color='platform')
fig3.update_layout(xaxis_title='Platform', yaxis_title='Total Sales (USD Million)')
fig3.show()

In Figure 3, it can be observed that the platforms with the all time highest total game sales are ps2 ($1233.56M), x360 ($961.24M), ps3 ($931.34M), wii ($891.18M), and ds ($802.76M). In Figure 2, it can be observed that all platforms are trending downward with three exceptions:
- The gen platform sales has an upward trend, but no data for this platform exists in the dataset past 1994
- The 3do platform sales has an upward trend, but no data for this platform exists in the dataset past 1995
- The gba platform sales has an slight upward trend, but no data for this platform exists in the dataset past 2007

Considering the trend observed of platforms rising and falling in popularity within a span of ten years, it can be assumed that the most popular gaming platform(s) of 2015-2025 is not included in this dataset. 

#### Are the differences in global sales of games for each platform significant? What about average sales on various platforms? 

In [65]:
# Build a box plot of game sale variation by platform
fig4 = px.box(df_filtered, x="platform", y="total_sales", hover_name="name", title='Figure 4: Game Sale Variation By Platform')
fig4.update_layout(xaxis_title='Game Platform', yaxis_title='Total Sales (USD Million)')
fig4.show()

Build a box plot for the global sales of all games, broken down by platform. Are the differences in sales significant? What about average sales on various platforms? Describe your findings.

#### Is there a correlation between user reviews and sales for the X platform?

#### How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?

#### Which genres are the most profitable? Which are the least?

In [62]:
# Create a df with the sum of all sales grouped by genre
genre_sales = df_filtered.groupby('genre')['total_sales'].sum().reset_index()

# Plot the total sales by genre
fig7 = px.histogram(genre_sales,
                    x='genre',
                    y='total_sales',
                    title='Figure 7: Total Sales by Genre',
                    color='genre')
fig7.update_layout(xaxis_title='Genre', yaxis_title='Total Sales (USD Million)')
fig7.show()

### Analysis by Sales Region
---
In this section, comparisons will be made between the three sales regions: North America, Europe, and Japan. The following factors will be analyzed:
- Platform market share variations by region
- Genre popularity variation across regions 
- Effect ESRB rating by region 

#### Platform Market Share Variations Across Regions

#### Genre Popularity Variations Across Regions

#### ESRB Rating Effect by Region 

### Hypothesis Testing 
---
This section will utilize hypothesis testing to see if the following two hypotheses are supported through statistical analysis:
1. Average user ratings of the Xbox One and PC platforms are the same. 
2. Average user ratings for the Action and Sports genres are different.

#### Are the average user ratings of the Xbox One and PC platforms the same?
- Null Hypothesis: There is a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.
- Alternative Hypothesis: There is not a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.

#### Are the average user ratings for the Action and Sports genres are different? 
- Null Hypothesis: There is not a statistcally significant difference between the average user ratings for the Action and Sports genres.
- Alternative Hypothesis: There is a statistcally significant difference between the average user ratings for the Action and Sports genres.



## Conclusions 
---